# Google Community Mobility Report (on COVID-19)

**DISCLAIMER**: This data is subject to change more often than the report structure and so most of the text will seen out of date if not viewed around the same time of creation of this report (12th May 2020).

Below is a short report and analysis of the Google Community Mobility Data for the UK. It makes use of the data provided by Google [here](https://www.google.com/covid19/mobility/) as well as data available through the Johns Hopkins [GitHub](https://www.google.com/covid19/mobility/).

In [39]:
import pandas
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

In [26]:
community_data = pandas.read_csv('https://raw.githubusercontent.com/nshyam97/Google-Community-Mobility-Data/master/UK_Global_Mobility_Report.csv')
community_data.head()

,country_region,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,United Kingdom,2020-02-15,-12.0,-7.0,-35.0,-12.0,-4.0,2.0
1,United Kingdom,2020-02-16,-7.0,-6.0,-28.0,-7.0,-3.0,1.0
2,United Kingdom,2020-02-17,10.0,1.0,24.0,-2.0,-14.0,2.0
3,United Kingdom,2020-02-18,7.0,-1.0,20.0,-3.0,-14.0,2.0
4,United Kingdom,2020-02-19,6.0,-2.0,8.0,-4.0,-14.0,3.0


Above is the data made available from the Google Community Mobility Report. The full csv file provided by Google includes a number of different countries and also within each country, sub regions. For the sake of this analysis and also for the storage limits applied by GitHub, I am just using the UK data as a whole rather than look into sub regions. I could look into sub regions at a later date to perform some comparisons with the country-wide data.

We can also see the different locations where we have mobility data and before we start the main part of the analysis, it would be important to define each of them according to how Google has classified them.

**Retail**: Places like restaurants, cafes, shopping centers, theme parks, museums, libraries, and movie theaters.

**Grocery**: Places like grocery markets, food warehouses, farmers markets, specialty food shops, drug stores, and pharmacies.

**Parks**: Places like national parks, public beaches, marinas, dog parks, plazas, and public gardens.

**Transit Stations**: Places like public transport hubs such as subway, bus, and train stations.

**Workplace**: Places of work.

**Residential**: Places of residence.

In [27]:
max(community_data.date)

'2020-05-07'

We can see that the dataset provides data from the 15th February 2020 till 2nd May 2020 at the time of writing (12th May 2020). Google outlines this in their website stating that the data is about 2-3 days behind which is why we will not have up-to-date data.

We can also see that we have a number of different areas in which we have mobility data which will allow us to see the difference between various industries.

First I'm going to clean the data slightly to make it easier to plot and understand. We need to change the column names to make them shorter and make the date column the index column.

In [28]:
community_data = community_data.set_index('date')
community_data = community_data.rename(columns={'retail_and_recreation_percent_change_from_baseline':'retail',
                      'grocery_and_pharmacy_percent_change_from_baseline':'grocery',
                      'parks_percent_change_from_baseline':'parks',
                      'transit_stations_percent_change_from_baseline':'transit-stations',
                      'workplaces_percent_change_from_baseline':'workplace',
                      'residential_percent_change_from_baseline':'residential'})
community_data.head()

,country_region,retail,grocery,parks,transit-stations,workplace,residential
date,,,,,,,
2020-02-15,United Kingdom,-12.0,-7.0,-35.0,-12.0,-4.0,2.0
2020-02-16,United Kingdom,-7.0,-6.0,-28.0,-7.0,-3.0,1.0
2020-02-17,United Kingdom,10.0,1.0,24.0,-2.0,-14.0,2.0
2020-02-18,United Kingdom,7.0,-1.0,20.0,-3.0,-14.0,2.0
2020-02-19,United Kingdom,6.0,-2.0,8.0,-4.0,-14.0,3.0


Firstly, just to show the data, I've plotted the percentage change in retail locations. We can see that there was a sharp rise in the days leading up to lockdown announcement and a further sharp drop on the day after the announcement was made. It then fluctuates depending on the day of the week but no apparent trend is seen on certain days.

In [35]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=community_data.index, y=community_data.retail, mode='lines'))
fig.update_layout(xaxis_title='Date', yaxis_title='Percentage Change (%)', 
                  title='Mobility Changes to Retail Locations')
fig.add_shape(dict(type = "line", y0 = 0, y1 = 1, yref = "paper",
            x0 = pandas.Timestamp('2020-03-23'), x1 = pandas.Timestamp('2020-03-23'), 
            line=dict(color="red", width=2)))
fig.add_shape(dict(type = "line", y0 = 0, y1 = 0,
            x0 = 0, x1 = 1, xref='paper', 
            line=dict(color="black", width=2, dash="dot")))
fig.add_annotation(x=pandas.Timestamp('2020-04-26'), y=5, xref="x", yref="y", text="Baseline", showarrow=False)
fig.add_annotation(x=pandas.Timestamp('2020-03-24'), y=-30, xref="x", yref="y", 
                   text="Lockdown announced", showarrow=True, ax=65, 
                  ay=-40, arrowhead=2, arrowsize=1)
fig.show()

We can take this a step further and compare this with other locations that we have available to us and see how things change then.

In [34]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=community_data.index, y=community_data.retail, mode='lines', name='Retail'))
fig.add_trace(go.Scatter(x=community_data.index, y=community_data.grocery, mode='lines', name='Grocery'))
fig.add_trace(go.Scatter(x=community_data.index, y=community_data.parks, mode='lines', name='Parks'))
fig.add_trace(go.Scatter(x=community_data.index, y=community_data['transit-stations'], mode='lines', name='Transit'))
fig.add_trace(go.Scatter(x=community_data.index, y=community_data.workplace, mode='lines', name='Workplace'))
fig.add_trace(go.Scatter(x=community_data.index, y=community_data.residential, mode='lines', name='Residential'))
fig.update_layout(xaxis_title='Date', yaxis_title='Percentage Change (%)', 
                  title='Mobility Changes to All Locations')
fig.add_shape(dict(type = "line", y0 = 0, y1 = 1, yref = "paper",
            x0 = pandas.Timestamp('2020-03-23'), x1 = pandas.Timestamp('2020-03-23'), 
            line=dict(color="red", width=2)))
fig.add_shape(dict(type = "line", y0 = 0, y1 = 0,
            x0 = 0, x1 = 1, xref='paper', 
            line=dict(color="black", width=2, dash="dot")))
fig.add_annotation(x=pandas.Timestamp('2020-04-26'), y=5, xref="x", yref="y", text="Baseline", showarrow=False)
fig.add_annotation(x=pandas.Timestamp('2020-03-24'), y=-30, xref="x", yref="y", 
                   text="Lockdown announced", showarrow=True, ax=65, 
                  ay=-40, arrowhead=2, arrowsize=1)
fig.show()

As we can see from the graph, we have a lot of lines for all the locations we have data for but we can see some interesting trends. For example, the parks seem to have reduced as the lockdown was announced but remained quite high up with roughly a -20-40% reduction from baseline. Most likely cause would be because the UK public were allowed to exercise once per day and so public parks were used for running or for walks. However, at the time of writing (13th May 2020), we have data up until 7th May 2020 and we can see that the use of parks has already increased over baseline to nearly +20%. 

On 10th May 2020, Boris Johnson announced the first stage of moving from level 4 to level 3 alert levels which are outlined in [this](https://order-order.com/wp-content/uploads/2020/05/FINAL-6.6637_CO_HMG_C19_Recovery_FINAL_110520_v1_WEB.pdf) 50 page document. One of the main points in this move towards level 3 is the ability to exercise an unlimited amount compared to just the 1 time before. This will inevitably cause the increase in mobility and number of people at parks but these rules only come into effect on 13th May 2020. The increase above baseline before the announcement was even made is worrying.

There seems to be a weird trend with the workplace mobility with an increase of people going to workplaces every Saturday and Sunday and droppping back down during the week. Google does not define the workplace further than "Place of work" so the range of workplaces could be vast and we would need more information to be able to see a reason why this could be happening.

We can look at the difference between Retail locations and Grocery locations as well to see how these compare.

In [37]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=community_data.index, y=community_data.retail, mode='lines', name='Retail'))
fig.add_trace(go.Scatter(x=community_data.index, y=community_data.grocery, mode='lines', name='Grocery'))
fig.update_layout(xaxis_title='Date', yaxis_title='Percentage Change (%)', 
                  title='Mobility Changes to Retail and Grocery Locations')
fig.add_shape(dict(type = "line", y0 = 0, y1 = 1, yref = "paper",
            x0 = pandas.Timestamp('2020-03-23'), x1 = pandas.Timestamp('2020-03-23'), 
            line=dict(color="red", width=2)))
fig.add_shape(dict(type = "line", y0 = 0, y1 = 0,
            x0 = 0, x1 = 1, xref='paper', 
            line=dict(color="black", width=2, dash="dot")))
fig.add_annotation(x=pandas.Timestamp('2020-04-26'), y=5, xref="x", yref="y", text="Baseline", showarrow=False)
fig.add_annotation(x=pandas.Timestamp('2020-03-24'), y=-30, xref="x", yref="y", 
                   text="Lockdown announced", showarrow=True, ax=65, 
                  ay=-40, arrowhead=2, arrowsize=1)
fig.show()

We can see that in the run up to the lockdown announcement that the retail locations reduce rapidly while the grocery locations see a sharp rise and fall before the lockdown. There was widespread news coverage due to many people hoarding toilet paper, handwash, hand sanitizers, and other household items in the run up to lockdown which would be a reason why the grocery locations saw a sharp rise.